In [1]:

##Extract
import random
import pyspark
from pyspark.sql import SparkSession, functions
import ConnectionConfig as cc
from pyspark.sql.functions import *

# Set up environment
cc.setupEnvironment()
cc.listEnvironment()
# Start local Spark cluster
spark = cc.startLocalCluster("stations_dim")
spark.getActiveSession() # To get the active Spark session
df_operational_stations_dim = (spark.read
    .format("jdbc")
    .option("driver", cc.get_Property("driver")) 
    .option("url", cc.create_jdbc()) 
    .option("dbtable", "stations") 
    .option("user", cc.get_Property("username")) 
    .option("password", cc.get_Property("password")) 
    .load())

# Show the schema and a few records to verify
df_operational_stations_dim.printSchema()
df_operational_stations_dim.show(5)




ALLUSERSPROFILE: C:\ProgramData
APPDATA: C:\Users\User\AppData\Roaming
COMMONPROGRAMFILES: C:\Program Files\Common Files
COMMONPROGRAMFILES(X86): C:\Program Files (x86)\Common Files
COMMONPROGRAMW6432: C:\Program Files\Common Files
COMPUTERNAME: MSI
COMSPEC: C:\WINDOWS\system32\cmd.exe
CONFIGSETROOT: C:\WINDOWS\ConfigSetRoot
DRIVERDATA: C:\Windows\System32\Drivers\DriverData
EFC_10468: 1
FPS_BROWSER_APP_PROFILE_STRING: Internet Explorer
FPS_BROWSER_USER_PROFILE_STRING: Default
HOMEDRIVE: C:
HOMEPATH: \Users\User
IPY_INTERRUPT_EVENT: 1732
JPY_INTERRUPT_EVENT: 1732
JPY_PARENT_PID: 1736
JPY_SESSION_NAME: DWH_StationDim.ipynb
LANG: en_US.UTF-8
LANGUAGE: 
LC_ALL: en_US.UTF-8
LOCALAPPDATA: C:\Users\User\AppData\Local
LOGONSERVER: \\MSI
NUMBER_OF_PROCESSORS: 20
ONEDRIVE: C:\Users\User\OneDrive
ONEDRIVECONSUMER: C:\Users\User\OneDrive
OS: Windows_NT
PATH: D:\KdG\Year 2\Integration\sparkdelta\venv\Scripts;C:\Program Files\Common Files\Oracle\Java\javapath;C:\Program Files (x86)\Common Files\Ora

In [2]:
#Transform
df_stations = spark.read \
    .format("jdbc") \
    .option("url", cc.create_jdbc()) \
    .option("driver", cc.get_Property("driver")) \
    .option("dbtable", "stations") \
    .option("user", cc.get_Property("username")) \
    .option("password", cc.get_Property("password")) \
    .load()

# TRANSFORM: Selecting required columns
df_result_stations = df_stations.select("stationid", "zipcode", "cityid", "gpscoord")

# Show the result to verify the selected columns
df_result_stations.show(5)


+---------+-------+------+-----------------+
|stationid|zipcode|cityid|         gpscoord|
+---------+-------+------+-----------------+
|        1|   2000|  NULL|(51.2182,4.41241)|
|        2|   2000|  NULL| (51.219,4.40405)|
|        3|   2000|  NULL|(51.2187,4.40066)|
|        4|   2000|  NULL|(51.2104,4.38772)|
|        5|   2018|  NULL|(51.2047,4.39625)|
+---------+-------+------+-----------------+
only showing top 5 rows



In [4]:
#Load
#Load
# Save the transformed data to a Delta table
spark.sql("DROP TABLE IF EXISTS stations_dim")

df_result_stations.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .option("path", "file:/D:/KdG/Year%202/Sem2/DataAI_Sem_2/sparkdelta/velo-warehouse") \
    .saveAsTable("stations_dim")

In [6]:
#Test
# Query the saved Delta table
spark.sql("SELECT * FROM stations_dim").show()


+---------+-------+------+-----------------+
|stationid|zipcode|cityid|         gpscoord|
+---------+-------+------+-----------------+
|        1|   2000|  NULL|(51.2182,4.41241)|
|        2|   2000|  NULL| (51.219,4.40405)|
|        3|   2000|  NULL|(51.2187,4.40066)|
|        4|   2000|  NULL|(51.2104,4.38772)|
|        5|   2018|  NULL|(51.2047,4.39625)|
|        6|   2018|  NULL|(51.1994,4.39013)|
|        7|   2000|  NULL|(51.2195,4.41164)|
|        8|   2018|  NULL|(51.2058,4.41836)|
|        9|   2018|  NULL|(51.2099,4.42248)|
|       10|   2140|  NULL|  (51.21,4.42741)|
|       11|   2060|  NULL|(51.2241,4.43216)|
|       12|   2060|  NULL|(51.2276,4.43923)|
|       13|   2050|  NULL| (51.2314,4.3787)|
|       14|   2050|  NULL| (51.228,4.37856)|
|       15|   2050|  NULL|(51.2184,4.38558)|
|       16|   2050|  NULL|  (51.22,4.37578)|
|       17|   2050|  NULL|(51.2206,4.38644)|
|       18|   2050|  NULL|(51.2334,4.38254)|
|       19|   2050|  NULL|(51.2231,4.38623)|
|       20

In [7]:
# Create a temporary view from the Delta table
spark.sql("CREATE OR REPLACE TEMP VIEW vehicle_dim_view AS SELECT * FROM stations_dim")


DataFrame[]